In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo
import json 
import attack
import matplotlib.pyplot as plt

import os

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)



2022-12-30 16:09:27.292564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X_raw =raw_data.drop('63', axis=1)
y_raw =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

Dataset:  ../datasets/dataset_purchase
Classes in classification task:  100


In [3]:
X_train, x_shadow, y_train, y_shadow = train_test_split(X_raw, y_raw, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")        


(39464, 600) (157859, 600)


/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_52662/3339399080.py:8: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_target_train = np.array(y_train[:X_train_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_52662/3339399080.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])


In [ ]:
rand_seed=2
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

model = algo.LogisticRegression_DPSGD()

model.n_classes      = n_classes
model.alpha          = 0.001
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)


In [ ]:
model.train_acc = model.evaluate(x_target_train, y_target_train, acc=True)
model.test_acc = model.evaluate(x_target_test, y_target_test, acc=True)

In [ ]:
model_dict = dict(model.__dict__)
model_dict['rand_seed'] = rand_seed
model_dict.pop('theta', None)
model_dict.pop('pred_func', None)
file_name = 'out_dp/baseline_tm'+''.join([str(i)+'_' for i in model_dict.values()])

In [ ]:
np.save(file_name + 'model', model.theta)
with open(file_name + 'params.json', 'w') as file:
    json.dump(model_dict, file)

In [ ]:
noise_model = algo.LogisticRegression_DPSGD()
noise_model.DP_out = True
noise_model.epsilon_out = 10
noise_model.delta_out = 10e-5
noise_model.lambda_ = 1e-1
noise_model.sensitivity = 2/(X_train_size*model.lambda_)  #lambda=.0001
    

In [ ]:
def set_model_params(model, params):

    model.n_classes      = params['n_classes']
    model.alpha          = params['alpha']
    model.max_iter       = params['max_iter']
    model.lambda_        = params['lambda_']
    model.tolerance      = params['tolerance']
    model.DP             = params['DP']
    model.L              = params['L']
    model.C              = params['C']
    model.epsilon        = params['epsilon']
    model.delta          = params['delta']

def output_DP(model,  X_train_size, epsilon_out_DP, delta_out_DP=1e-5):
    #gaussian mechanism 
    noise_model = algo.LogisticRegression_DPSGD()
    set_model_params(noise_model, target_model.__dict__)
    
    noise_model.DP_out = True
    noise_model.epsilon_out = epsilon_out_DP
    noise_model.delta_out = delta_out_DP
    noise_model.sensitivity = 2/(X_train_size*model.lambda_) 

    sigma = np.sqrt(2 * np.log(1.25 / noise_model.delta_out)) * (noise_model.sensitivity / noise_model.epsilon_out)

    ns=np.random.normal(loc=0.0, scale=sigma, size=model.theta.shape)

    noise_model.theta = model.theta + ns
    
    return noise_model


In [6]:
file = 'tm/rs42_lr0.001_iter100_reg0.005_DPFalse_target_model_params.json'
with open(file) as json_file:
    tm_params = json.load(json_file)
            
x_target_train, y_target_train, x_target_test, y_target_test = attack.data_shuffle(rand_seed, X_raw, y_raw)
target_model = algo.LogisticRegression_DPSGD()
scripts.set_model_params(target_model, tm_params)
target_model.theta = np.load('tm/rs42_lr0.001_iter100_reg0.005_DPFalse_target_model.npy')
target_model.evaluate(x_target_train, y_target_train, acc=True)
target_model.evaluate(x_target_test, y_target_test, acc=True)
 

The accuracy of the model : 82.69999999999999 %
The accuracy of the model : 52.6 %


0.5255

In [4]:
import scripts

In [7]:
noise_model = scripts.output_DP(target_model, epsilon_out_DP=10, delta_out_DP=1e-5, X_train_size=X_train_size)
noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=True)
noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test, acc=True)
    

The accuracy of the model : 77.5 %
The accuracy of the model : 48.6 %


In [ ]:
target_model.evaluate(x_target_train, y_target_train, acc=True)
target_model.evaluate(x_target_test, y_target_test, acc=True)
 

In [ ]:
noise_model = algo.LogisticRegression_DPSGD()
noise_model.DP_out = True
noise_model.epsilon_out = 10
noise_model.delta_out = 10e-5
noise_model.lambda_ = 1e-1
noise_model.sensitivity = 2/(X_train_size*model.lambda_)  #lambda=.0001
    
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

sigma = np.sqrt(2 * np.log(1.25 / noise_model.delta_out)) * (noise_model.sensitivity / noise_model.epsilon_out)

ns=np.random.normal(loc=0.0, scale=sigma, size=model.theta.shape)

noise_model.theta = model.theta + ns
noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=True)
noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test, acc=True)

ns

In [ ]:
new_dict = dict(noise_model.__dict__)
new_dict['rand_seed'] = rand_seed
new_dict.pop('theta', None)
file_name = 'out_dp/output_dp'+''.join([str(i)+'_' for i in new_dict.values()])

In [ ]:
np.save(file_name + 'model', model.theta)
with open(file_name + 'params.json', 'w') as file:
    json.dump(new_dict, file)

In [ ]:
# out_eps = [0.1, 0.5, 1, 5, 10, 50, 100,500, 1000, 5000, 10000]
out_eps = [0.001, 0.005, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,5,10,50,100,200,500,1000]
lambdas = [1000, 100, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
for rand_seed in [2, 4, 23]:
    for eps in out_eps:
        for lam in lambdas:
            params_list.append((rand_seed,eps, lam))
            
for rand_seed, eps, lam in params_list:

    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)

    noise_model = algo.LogisticRegression_DPSGD()
    noise_model.DP_out = True
    noise_model.epsilon_out = eps
    noise_model.delta_out = 10e-5
    noise_model.lambda_ = lam
    noise_model.sensitivity = 2/(X_train_size*noise_model.lambda_)  #lambda=.0001
     
    #gaussian mechanism 
    sigma = np.sqrt(2 * np.log(1.25 / noise_model.delta_out)) * (noise_model.sensitivity / noise_model.epsilon_out)
    ns=np.random.normal(loc=0.0, scale=sigma, size=model.theta.shape)

    noise_model.theta = model.theta + ns
    noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=True)
    noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test, acc=True)

    new_dict = dict(noise_model.__dict__)
    new_dict['rand_seed'] = rand_seed
    new_dict.pop('theta', None)
    file_name = 'out_dp/output_dp'+''.join([str(i)+'_' for i in new_dict.values()])
    
    np.save(file_name + 'model', model.theta)
    with open(file_name + 'params.json', 'w') as file:
        json.dump(new_dict, file)

In [ ]:
out_eps = [0.1, 0.5, 1, 5, 10, 50, 100,500, 1000, 5000, 10000]
lambdas = [1000, 100, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
params_list = []
for rand_seed in [1,13, 42]:
    for eps in out_eps:
        for lam in lambdas:
            params_list.append((rand_seed,eps, lam))

In [ ]:
for rand_seed, eps, lam in params_list:
    print(rand_seed, eps, lam)

# Output DP


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import algo
import json

In [ ]:
path = 'tm'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if 'DPFalse' in file and 'lr0.001_iter100_' in file:
            if '.npy' in file:
                tms[file.replace('.npy','')] = np.load(path+'/'+file)
            elif '.json' in file:
                with open(path+'/'+file) as json_file:
                    tms_params[file.replace('_params.json','')]= json.load(json_file)
            


In [ ]:
df = pd.DataFrame.from_dict(tms_params, orient='index')
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'file_name'})
# df

In [ ]:
rs_res = {}

In [ ]:
rand_seed=42 

rs_res[rand_seed] = []
for i in tms:
    if 'rs'+str(rand_seed)+'_' in i:
        rs_res[rand_seed].append(i)

In [ ]:

np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

X_train, x_shadow, y_train, y_shadow = train_test_split(X_raw, y_raw, train_size=0.2, random_state=rand_seed)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")        



In [ ]:
# for i in rs_res[rand_seed]:
#     model = algo.LogisticRegression_DPSGD()
#     model.n_classes  = n_classes

#     model.theta = tms[i]
#     model.train_acc = model.evaluate(x_target_train, y_target_train, acc=True)
#     model.test_acc = model.evaluate(x_target_test, y_target_test, acc=True)
# #     print(df.loc[df['file_name']==i]['test_acc'])

In [ ]:
# out_eps = [0.1, 0.5, 1, 5, 10, 50, 100,500, 1000]
# out_eps = [0.1, 0.3, 0.5, 0.7, 1, 5, 10, 30 , 50 , 70, 100] 
out_eps = [i/10 for i in range(1,10)] + [i for i in range(1,11)] + [i*10 for i in range(2,11)] + [i*100 for i in range(2,11)]+ [i*1000 for i in range(2,11)] + [i*10000 for i in range(2,11)]
model = {}
noised_models = {}
results_test = {}
results_train = {}
output_dp_res = {}
for key in rs_res[rand_seed]:

    model['theta'] = tms[key]
    model['lambda_'] = df.loc[df['file_name']==key]['lambda_'].item()

    results_test[model['lambda_']] = []
    results_train[model['lambda_']] = []
    noised_models[model['lambda_']] = {}
    
    for eps in out_eps:
        
        if os.path.exists(f'out_dp/{key}_outEps{eps}params.json'):
            continue
        
        noise_model = algo.LogisticRegression_DPSGD()
        noise_model.DP_out = True
        noise_model.epsilon_out = eps
        noise_model.delta_out = 10e-5
        noise_model.sensitivity = 2/(X_train_size*model['lambda_'])  #lambda=.0001

        #gaussian mechanism 
        sigma = np.sqrt(2 * np.log(1.25 / noise_model.delta_out)) * (noise_model.sensitivity / noise_model.epsilon_out)
        ns=np.random.normal(loc=0.0, scale=sigma, size=model['theta'].shape)

        noise_model.theta = model['theta'] + ns
        noised_models[model['lambda_']][eps] = noise_model.theta
        output_dp_res[f'{key}_outEps{eps}'] = tms_params[key]
        output_dp_res[f'{key}_outEps{eps}']['outDP_epsilon'] = eps
        noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=False)
        noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test, acc=False)
        
        output_dp_res[f'{key}_outEps{eps}']['outDP_train_acc'] = noise_model.train_acc
        output_dp_res[f'{key}_outEps{eps}']['outDP_test_acc'] = noise_model.test_acc
        
        results_train[model['lambda_']].append(noise_model.train_acc)
        results_test[model['lambda_']].append(noise_model.test_acc)
        
        np.save(f'out_dp/{key}_outEps{eps}', noise_model.theta)
        with open(f'out_dp/{key}_outEps{eps}params.json', 'w') as file:
            json.dump(output_dp_res[f'{key}_outEps{eps}'], file)

In [ ]:
sort_keys = list(results_test.keys())
sort_keys.sort()
sort_keys

In [ ]:
# l2 reg and target model accuracy with output DP

for key in sort_keys:
    
    if max(results_test[key])>0.:
        print(key)
        plt.plot(out_eps, results_test[key], label='l2reg: '+str(key))
#         print(lambda_res['epsilon_out'])
#         plt.errorbar(lambda_res['epsilon_out'], lambda_res['train_acc_mean'], lambda_res['train_acc_std'], label='lam'+str(lam)+' train')  
#         plt.errorbar(lambda_res['epsilon_out'], lambda_res['test_acc_mean'], lambda_res['test_acc_std'], label='lam'+str(lam)+' test')
        plt.fill_between(out_eps, results_train[key], results_test[key], alpha=0.4,) #label='lam'+str(lam))
# plt.legend(loc='lower right')
# plt.xlabel('epsilon')
# plt.ylabel('accuracy')   
# plt.savefig('figures/outputDP_train_test_lambda_epsilon100_all.png')
plt.plot([out_eps[0],out_eps[-1]], [0.56, 0.56], label='noDP baseline')

plt.legend(loc='lower right', bbox_to_anchor=(0.27, 0.6), fontsize=8)
plt.xlabel('epsilon')
plt.ylabel('accuracy')   
plt.xscale('log')
# plt.title('Target model accuracy in output DP centralised with different l2 reg')
# plt.savefig('figures/outputDP_l2_best_target_model_epsilon_labda.png')

In [ ]:
data_save = {}
for key in sort_keys:
    
    data_save['epsilon'] = out_eps
    data_save[f'{key}l2_train_acc'] = results_train[key]
    data_save[f'{key}l2_test_acc'] = results_test[key]

    

In [ ]:
# # data_save['epsilon'] 
# d = pd.DataFrame.from_dict(data_save)
# # d.index = d.index.rename('epsilon')
# d.to_csv('figures/outputDP_target_model_epsilon_labda.csv')
# # d

# FL output DP

In [ ]:
path = 'fl'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            for k in results[r]['Unnamed: 0']:
                it.append(k[k.find('i')+1:k.find('_')])
                client.append(k[k.find('_')+1:])
            results[r]['it'] = it
            results[r]['client'] = client
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)



In [ ]:
file = 'fl/rs42_ncl2_fiter5_lr0.01_iter50_reg0.005_DPFalse'
# file = 'fl/rs42_ncl4_fiter5_lr0.01_iter50_reg1e-05_DPFalse'
# file = 'fl/rs42_ncl8_fiter5_lr0.001_iter300_reg1e-05_DPFalse'
nodes = int(file[file.find('_ncl')+4:file.find('_fiter')])

In [ ]:
fl_models = {}
for i in models[file]:
    if 'i0_' in i:
        fl_models[i[i.find('i0_c')+4:i.find('.npy')]] = models[file][i]

In [ ]:

for m in fl_models:
    model = algo.LogisticRegression_DPSGD()
    model.theta = fl_models[m]
    model.test_acc = model.evaluate(x_target_test, y_target_test, acc=True)
    nodp_global = model.test_acc

In [ ]:
out_eps = [i/10 for i in range(1,10)] + [i for i in range(1,11)] #+ [i*10 for i in range(2,11)] 

fl_noised_eps = {}
for eps in out_eps :
    fl_noised_eps[eps] = {}
    for m in fl_models:
        model = algo.LogisticRegression_DPSGD()
        model.theta = fl_models[m]

        noise_model = algo.LogisticRegression_DPSGD()
        noise_model.DP_out = True
        noise_model.epsilon_out = eps
        noise_model.delta_out = 10e-5
        noise_model.sensitivity = 2/(X_train_size*.005)#params[file]['lambda_'])  #lambda=.005

        #gaussian mechanism 
        sigma = np.sqrt(2 * np.log(1.25 / noise_model.delta_out)) * (noise_model.sensitivity / noise_model.epsilon_out)
        ns=np.random.normal(loc=0.0, scale=sigma, size=model.theta.shape)

        noise_model.theta = model.theta + ns
    #     noise_model.train_acc = noise_model.evaluate(x_target_train, y_target_train, acc=True)
        noise_model.test_acc = noise_model.evaluate(x_target_test, y_target_test)
        fl_noised_eps[eps][m] = noise_model

    gm = fl_noised_eps[eps]['0'].theta
    for n, m in enumerate(fl_noised_eps[eps]):
        if m != '0' and m != 'g':
            gm += fl_noised_eps[eps][m].theta

    fl_noised_eps[eps]['g'] = algo.LogisticRegression_DPSGD()
    fl_noised_eps[eps]['g'].theta = gm/n
    fl_noised_eps[eps]['g'].test_acc = fl_noised_eps[eps]['g'].evaluate(x_target_test, y_target_test)

In [ ]:
results = {}
for eps in fl_noised_eps:
    results[eps] = []
    for m in fl_noised_eps[eps]:
        results[eps].append(fl_noised_eps[eps][m].test_acc)

In [ ]:
plt.plot([out_eps[0],out_eps[-1]], [0.56, 0.56], label='noDP centr baseline')
plt.plot([out_eps[0],out_eps[-1]], [ nodp_global,  nodp_global], label='noDP fl global baseline')

data = pd.DataFrame.from_dict(results, orient='index')
for i, m in enumerate(fl_noised_eps[eps]):
    plt.plot(data.index, data[i], label=f'client {m}')

plt.legend(loc='lower right', bbox_to_anchor=(1, 0), fontsize=8)
plt.xlabel('epsilon')
plt.ylabel('accuracy')   
plt.title(f'Output DP Federated {nodes} nodes')
# plt.savefig(f'figures/fl/outputDP_{nodes}nodes.png')
# data.to_csv(f'figures/fl/outputDP_{nodes}nodes.csv')


In [ ]:
from torch import nn, optim

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

def test_mi_attack(attack_models, target_model, x_target_train, y_target_train, x_target_test, y_target_test):
    
    results = {}
    at_acc = []
    at_rec = []
    at_pre = []
    for am in attack_models:
        a_model = attack_models[am] 
        attack_acc, attack_pre, attack_rec = attack.mi_attack_test(target_model, a_model, x_target_train, y_target_train, x_target_test, y_target_test)
        at_acc.append(attack_acc)
        at_pre.append(attack_pre)
        at_rec.append(attack_rec)
    results['attack_acc_mean'] = np.mean(at_acc)
    results['attack_acc_std'] = np.std(at_acc)
    results['attack_pre_mean'] = np.mean(at_pre)
    results['attack_pre_std'] = np.std(at_pre)
    results['attack_rec_mean'] = np.mean(at_rec)
    results['attack_rec_std'] = np.std(at_rec)
    
    return results
path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

In [ ]:
x_target_train = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
y_target_train = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
x_target_test = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
y_target_test = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
n_classes = len(np.unique(y_target_train))

number_of_clients = nodes
data_per_client = int(x_target_train.shape[0]/number_of_clients)

clients = {}
for i in range(number_of_clients):
    
    clients[str(i)] = algo.LogisticRegression_DPSGD()
    clients[str(i)].n_classes = n_classes
    
    clients[str(i)].x = x_target_train[i*data_per_client:(i+1)*data_per_client]
    clients[str(i)].y = y_target_train[i*data_per_client:(i+1)*data_per_client]
    clients[str(i)].theta = fl_models[str(i)]

clients['g'] = algo.LogisticRegression_DPSGD()
clients['g'].n_classes = n_classes
clients['g'].theta = fl_models['g']
clients['g'].x = x_target_train
clients['g'].y = y_target_train

In [ ]:
mi_results = {}
attack_acc = []
for eps in fl_noised_eps:
    mi_results[eps] = {}
    for i in fl_noised_eps[eps]:
        fl_noised_eps[eps][i].n_classes = n_classes
        mi_results[eps][i] = attack.test_mi_attack(ams, fl_noised_eps[eps][i], clients[i].x, clients[i].y, x_target_test, y_target_test)

In [ ]:
attack_acc = {}
for eps in fl_noised_eps:
    attack_acc[eps] = []
    for i in fl_noised_eps[eps]:
        attack_acc[eps].append(mi_results[eps][i]['attack_acc_mean'])

In [ ]:
plt.plot([out_eps[0],out_eps[-1]], [0.655836, 0.655836], label='noDP centr MI baseline')
# plt.plot([out_eps[0],out_eps[-1]], [ nodp_global,  nodp_global], label='noDP fl global baseline')

data = pd.DataFrame.from_dict(attack_acc, orient='index')
for i, m in enumerate(fl_noised_eps[eps]):
    plt.plot(data.index, data[i], label=f'client {m}')
    plt.plot([out_eps[0],out_eps[-1]], [fl_baseline[nodes][m]['attack_acc_mean'],fl_baseline[nodes][m]['attack_acc_mean']], label=f'client {m} no DP')

plt.legend(loc='lower right', bbox_to_anchor=(1, 0), fontsize=8)
plt.xlabel('epsilon')
plt.ylabel('accuracy')   
plt.title(f'Membershi Inference Output DP Federated {nodes} nodes')
# plt.savefig(f'figures/fl/MI_outputDP_{nodes}nodes.png')
# data.to_csv(f'figures/fl/MI_outputDP_{nodes}nodes.csv')


In [ ]:
fl_baseline = {}

In [ ]:
fl_baseline[nodes][m]

In [ ]:
fl_baseline[2] = {'0': {'attack_acc_mean': 0.6645666666666666,
  'attack_acc_std': 0.0028941191969104794,
  'attack_pre_mean': 0.3725942915392457,
  'attack_pre_std': 0.041945700055508545,
  'attack_rec_mean': 0.005966666666666666,
  'attack_rec_std': 0.008488750726042608},
 '1': {'attack_acc_mean': 0.6645333333333333,
  'attack_acc_std': 0.0030871297948693783,
  'attack_pre_mean': 0.4379919989676087,
  'attack_pre_std': 0.08777678967598833,
  'attack_rec_mean': 0.005866666666666667,
  'attack_rec_std': 0.0077645919975803555},
 'g': {'attack_acc_mean': 0.4998916666666666,
  'attack_acc_std': 0.00042270622843240793,
  'attack_pre_mean': 0.5766793409378961,
  'attack_pre_std': 0.06930078663902585,
  'attack_rec_mean': 0.005916666666666666,
  'attack_rec_std': 0.008117145776413995}}

In [ ]:
fl_baseline[4] = {'0': {'attack_acc_mean': 0.7827466666666667,
  'attack_acc_std': 0.02855516921485302,
  'attack_pre_mean': 0.12404040674785337,
  'attack_pre_std': 0.15032659664100007,
  'attack_rec_mean': 0.17113333333333336,
  'attack_rec_std': 0.36985246908583547},
 '1': {'attack_acc_mean': 0.7826533333333333,
  'attack_acc_std': 0.029852058927689076,
  'attack_pre_mean': 0.33998973470304833,
  'attack_pre_std': 0.090603241237165,
  'attack_rec_mean': 0.17353333333333332,
  'attack_rec_std': 0.3693644571723465},
 '2': {'attack_acc_mean': 0.7819600000000001,
  'attack_acc_std': 0.031078180984950424,
  'attack_pre_mean': 0.38269610973773016,
  'attack_pre_std': 0.13809448542080138,
  'attack_rec_mean': 0.17253333333333334,
  'attack_rec_std': 0.3688802997661376},
 '3': {'attack_acc_mean': 0.78232,
  'attack_acc_std': 0.029695557467967072,
  'attack_pre_mean': 0.1288500866036267,
  'attack_pre_std': 0.15065452542844787,
  'attack_rec_mean': 0.17153333333333332,
  'attack_rec_std': 0.369677583247288},
 'g': {'attack_acc_mean': 0.5271083333333334,
  'attack_acc_std': 0.060952880822448055,
  'attack_pre_mean': 0.5744581818345609,
  'attack_pre_std': 0.06791620590175466,
  'attack_rec_mean': 0.15499999999999997,
  'attack_rec_std': 0.3310287298709887}}

In [ ]:
fl_baseline[8] = {'0': {'attack_acc_mean': 0.8910518518518519,
  'attack_acc_std': 0.019347286011079507,
  'attack_pre_mean': 0.12887480893856348,
  'attack_pre_std': 0.2247570112117821,
  'attack_rec_mean': 0.1636,
  'attack_rec_std': 0.3554892966039906},
 '1': {'attack_acc_mean': 0.8923703703703705,
  'attack_acc_std': 0.021520195000671836,
  'attack_pre_mean': 0.14358680674134058,
  'attack_pre_std': 0.23247423509235346,
  'attack_rec_mean': 0.16466666666666666,
  'attack_rec_std': 0.3529225536699077},
 '2': {'attack_acc_mean': 0.892725925925926,
  'attack_acc_std': 0.02178439304918229,
  'attack_pre_mean': 0.27626076164377017,
  'attack_pre_std': 0.17979309640432037,
  'attack_rec_mean': 0.16533333333333333,
  'attack_rec_std': 0.35015293166780076},
 '3': {'attack_acc_mean': 0.8929777777777779,
  'attack_acc_std': 0.02254468265577016,
  'attack_pre_mean': 0.2738904422528505,
  'attack_pre_std': 0.1855714981246906,
  'attack_rec_mean': 0.1650666666666667,
  'attack_rec_std': 0.3513210624043041},
 '4': {'attack_acc_mean': 0.8931703703703704,
  'attack_acc_std': 0.022875172685189273,
  'attack_pre_mean': 0.27598034224776385,
  'attack_pre_std': 0.1843304074562958,
  'attack_rec_mean': 0.16733333333333333,
  'attack_rec_std': 0.35567668214576875},
 '5': {'attack_acc_mean': 0.8933037037037037,
  'attack_acc_std': 0.023586767116126724,
  'attack_pre_mean': 0.3437877398523385,
  'attack_pre_std': 0.20216409307432917,
  'attack_rec_mean': 0.16453333333333334,
  'attack_rec_std': 0.35257911956829713},
 '6': {'attack_acc_mean': 0.8929185185185187,
  'attack_acc_std': 0.022709598654671145,
  'attack_pre_mean': 0.14686515137505105,
  'attack_pre_std': 0.2363595774603635,
  'attack_rec_mean': 0.166,
  'attack_rec_std': 0.35588994553560144},
 '7': {'attack_acc_mean': 0.8940444444444445,
  'attack_acc_std': 0.025480585686235305,
  'attack_pre_mean': 0.1460447908388099,
  'attack_pre_std': 0.2505324248748163,
  'attack_rec_mean': 0.1633333333333333,
  'attack_rec_std': 0.3527641453184014},
 'g': {'attack_acc_mean': 0.5195083333333333,
  'attack_acc_std': 0.04395930330683396,
  'attack_pre_mean': 0.5726381161688232,
  'attack_pre_std': 0.06703511936618912,
  'attack_rec_mean': 0.12289999999999997,
  'attack_rec_std': 0.25927604980020813}}